In [14]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 14: Regolith Reservoir ---</h2><p>The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads <em>behind</em> the waterfall.</p>
<p>Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.</p>
<p>As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!</p>
<p>Fortunately, your <a href="/2018/day/17">familiarity</a> with analyzing the path of falling material will come in handy here. You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly <em>air</em> with structures made of <em>rock</em>.</p>
<p>Your scan traces the path of each solid rock structure and reports the <code>x,y</code> coordinates that form the shape of the path, where <code>x</code> represents distance to the right and <code>y</code> represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:</p>
<pre><code>498,4 -&gt; 498,6 -&gt; 496,6
503,4 -&gt; 502,4 -&gt; 502,9 -&gt; 494,9
</code></pre>
<p>This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from <code>498,4</code> through <code>498,6</code> and another line of rock from <code>498,6</code> through <code>496,6</code>.)</p>
<p>The sand is pouring into the cave from point <code>500,0</code>.</p>
<p>Drawing rock as <code>#</code>, air as <code>.</code>, and the source of the sand as <code>+</code>, this becomes:</p>
<pre><code>
  4     5  5
  9     0  0
  4     0  3
0 ......+...
1 ..........
2 ..........
3 ..........
4 ....#...##
5 ....#...#.
6 ..###...#.
7 ........#.
8 ........#.
9 #########.
</code></pre>
<p>Sand is produced <em>one unit at a time</em>, and the next unit of sand is not produced until the previous unit of sand <em>comes to rest</em>. A unit of sand is large enough to fill one tile of air in your scan.</p>
<p>A unit of sand always falls <em>down one step</em> if possible. If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally <em>one step down and to the left</em>. If that tile is blocked, the unit of sand attempts to instead move diagonally <em>one step down and to the right</em>. Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. If all three possible destinations are blocked, the unit of sand <em>comes to rest</em> and no longer moves, at which point the next unit of sand is created back at the source.</p>
<p>So, drawing sand that has come to rest as <code>o</code>, the first unit of sand simply falls straight down and then stops:</p>
<pre><code>......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......<em>o</em>.#.
#########.
</code></pre>
<p>The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:</p>
<pre><code>......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.
</code></pre>
<p>After a total of five units of sand have come to rest, they form this pattern:</p>
<pre><code>......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.
</code></pre>
<p>After a total of 22 units of sand:</p>
<pre><code>......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.
</code></pre>
<p>Finally, only two more units of sand can possibly come to rest:</p>
<pre><code>......+...
..........
......o...
.....ooo..
....#ooo##
...<em>o</em>#ooo#.
..###ooo#.
....oooo#.
.<em>o</em>.ooooo#.
#########.
</code></pre>
<p>Once all <code><em>24</em></code> units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with <code>~</code>:</p>
<pre><code>.......+...
.......~...
......~o...
.....~ooo..
....~#ooo##
...~o#ooo#.
..~###ooo#.
..~..oooo#.
.~o.ooooo#.
~#########.
~..........
~..........
~..........
</code></pre>
<p>Using your scan, simulate the falling sand. <em>How many units of sand come to rest before sand starts flowing into the abyss below?</em></p>
</article>


In [15]:
from itertools import zip_longest
from more_itertools import flatten, minmax


tests = [
    {
        "name": "Example",
        "s": """
            498,4 -> 498,6 -> 496,6
            503,4 -> 502,4 -> 502,9 -> 494,9
        """,
        "expected": 24,
    },
]


class Drawing:
    def __init__(self, s: str) -> None:
        lines = [
            [tuple(map(int, p.split(","))) for p in l.strip().split(" -> ")]
            for l in s.strip().splitlines()
        ]

        self.grid = {}

        for linesegments in lines:
            for i in range(1, len(linesegments)):
                fr_x, fr_y = linesegments[i - 1]
                to_x, to_y = linesegments[i]
                fr_x, to_x = minmax(fr_x, to_x)
                fr_y, to_y = minmax(fr_y, to_y)

                self.grid[(500, 0)] = "+"

                for y, x in product(range(fr_y, to_y + 1), range(fr_x, to_x + 1)):
                    self.grid[(x, y)] = "#"

    def drop_sand(self) -> Drawing:
        count = 0
        while not self.dropped_unit_falls_throug():
            count += 1

        return count

    def dropped_unit_falls_throug(self) -> bool:
        x_min, x_max = minmax(x for x, _ in self.grid.keys())
        y_min, y_max = minmax(y for _, y in self.grid.keys())
        x, y = 500, 0

        while x_min <= x <= x_max and y_min <= y <= y_max:
            if (x, y + 1) not in self.grid:
                y += 1
            elif (x - 1, y + 1) not in self.grid:
                x -= 1
                y += 1
            elif (x + 1, y + 1) not in self.grid:
                x += 1
                y += 1
            else:
                self.grid[(x, y)] = "o"
                return False

        return True

    def __str__(self) -> str:
        fr_x, to_x = minmax(x for x, _ in self.grid.keys())
        fr_y, to_y = minmax(y for _, y in self.grid.keys())

        grid = [["."] * (to_x - fr_x + 1) for _ in range(fr_y, to_y + 1)]

        for (x, y), v in self.grid.items():
            grid[y - fr_y][x - fr_x] = v

        return "\n".join(
            (
                "\n".join(
                    f"     {"".join(self.header(x, t, fr_x,to_x) for x in range(fr_x, to_x + 1))}"
                    for t in zip_longest(str(fr_x), "500", str(to_x))
                ),
                "\n".join(f'{i:4d} {"".join(l)}' for i, l in enumerate(grid)),
            )
        )

    def header(self, x, t, fr_x, to_x) -> str:
        x_range = to_x - fr_x + 1
        x500 = 500 - fr_x

        if x == 0:
            return t[0] or " "
        if x == x500:
            return t[1] or " "
        if x == x_range - 1:
            return t[-1] or " "
        return " "

    __repr__ = __str__


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Drawing(s).drop_sand()


Test Example passed, for test_part_I.
Success


In [16]:
with open("../input/day14.txt") as f:
    puzzle = f.read()

print(f"Part I: {Drawing(puzzle).drop_sand()}")

Part I: 817


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>817</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You realize you misread the scan. There isn't an <span title="Endless Void is my C cover band.">endless void</span> at the bottom of the scan - there's floor, and you're standing on it!</p>
<p>You don't have time to scan the floor, so assume the floor is an infinite horizontal line with a <code>y</code> coordinate equal to <em>two plus the highest <code>y</code> coordinate</em> of any point in your scan.</p>
<p>In the example above, the highest <code>y</code> coordinate of any point is <code>9</code>, and so the floor is at <code>y=11</code>. (This is as if your scan contained one extra rock path like <code>-infinity,11 -&gt; infinity,11</code>.) With the added floor, the example above now looks like this:</p>
<pre><code>        ...........+........
        ....................
        ....................
        ....................
        .........#...##.....
        .........#...#......
        .......###...#......
        .............#......
        .............#......
        .....#########......
        ....................
&lt;-- etc #################### etc --&gt;
</code></pre>
<p>To find somewhere safe to stand, you'll need to simulate falling sand until a unit of sand comes to rest at <code>500,0</code>, blocking the source entirely and stopping the flow of sand into the cave. In the example above, the situation finally looks like this after <code><em>93</em></code> units of sand come to rest:</p>
<pre><code>............o............
...........ooo...........
..........ooooo..........
.........ooooooo.........
........oo#ooo##o........
.......ooo#ooo#ooo.......
......oo###ooo#oooo......
.....oooo.oooo#ooooo.....
....oooooooooo#oooooo....
...ooo#########ooooooo...
..ooooo.......ooooooooo..
#########################
</code></pre>
<p>Using your scan, simulate the falling sand until the source of the sand becomes blocked. <em>How many units of sand come to rest?</em></p>
</article>


In [17]:
tests = [
    {
        "name": "Example",
        "s": """
            498,4 -> 498,6 -> 496,6
            503,4 -> 502,4 -> 502,9 -> 494,9
        """,
        "expected": 93,
    },
]


class FlooredDrawing(Drawing):
    def __init__(self, s) -> None:
        super().__init__(s)

        self.floor = max(y for _, y in self.grid.keys()) + 2
        x_min, x_max = minmax(x for x, _ in self.grid.keys())

        for x in range(x_min, x_max + 1):
            self.grid[(x, self.floor)] = "#"

    def dropped_unit_falls_throug(self) -> bool:
        x_min, x_max = minmax(x for x, _ in self.grid.keys())
        y_min, y_max = minmax(y for _, y in self.grid.keys())
        x, y = 500, 0

        if self.grid[(x, y)] == "o":
            return True

        while True:
            if y + 1 < self.floor and (x, y + 1) not in self.grid:
                y += 1
            elif y + 1 < self.floor and (x - 1, y + 1) not in self.grid:
                x -= 1
                y += 1
            elif y + 1 < self.floor and (x + 1, y + 1) not in self.grid:
                x += 1
                y += 1
            else:
                self.grid[(x, y)] = "o"
                return False


@test(tests=tests)
def test_part_I(s: str) -> int:
    return FlooredDrawing(s).drop_sand()


Test Example passed, for test_part_I.
Success


In [18]:
print(f"Part II: { FlooredDrawing(puzzle).drop_sand()}")

Part II: 23416


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>23416</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
